<h1> CV 2023 </h1>

In [10]:
# from google.colab import drive
# drive.mount('/content/drive')

In [11]:
print("CV2023")

CV2023




*   integrator erklären +Bilder + focal planes
*   model impainting
* Diagramm vom Model
* pre processing
*





<h1> Encoder - Decoder Implementation </h1>

To run the code:


*   Download the zip file from WeTransfer
*   Either mount drive to Google Colab and upload images there
*   or upload to content folder here (will only be for this instance)
*   change strings to files accordingly


In [12]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import torch.nn as nn
from tqdm import tqdm

In [13]:
class InpaintingDataset(Dataset):
    def __init__(self, input_root, target_root, params_root):
        self.input_root = input_root
        self.target_root = target_root
        self.params_root = params_root
        self.input_list = os.listdir(input_root)
        self.target_list = os.listdir(target_root)

    def __len__(self):
        return len(self.target_list)

    def __getitem__(self, idx):

        input_base_name = os.path.join(self.input_root, self.input_list[idx*6])
        target_name = os.path.join(self.target_root, self.target_list[idx])
        parts = os.path.basename(target_name).split('_')
        params_name = os.path.join(self.params_root, parts[0] + "_" + parts[1] + "_Parameters.txt")  # Assuming filenames match
        # Stack multiple input images with similar names
        input_images = []
        for suffix in ['0', '0.5', '1', '1.5', '2', '2.5']:
            input_name = os.path.join(self.input_root, f"{parts[0]}_{parts[1]}_{suffix}_integral.png")
            input_image = Image.open(input_name).convert("L")
            input_image = transforms.ToTensor()(input_image)
            input_images.append(input_image)

        # Stack images along a new dimension
        stacked_images = torch.cat(input_images, dim=0)
        target_image = Image.open(target_name).convert("L")

        target_image = transforms.ToTensor()(target_image)

        # Load and process additional parameters
        with open(params_name, 'r') as file:
            params_content = file.read()

        # Add additional parameters to the tuple
        return stacked_images, target_image, params_content

In [14]:
# Specify your dataset paths for input and target images
input_images_path = os.path.join("cnn_test_input", "x")
target_images_path = os.path.join("cnn_test_input", "y")
params_path = os.path.join("cnn_test_input", "params")

# Create datasets
dataset = InpaintingDataset(input_root=input_images_path, target_root=target_images_path, params_root=params_path)

In [15]:
# Create dataloaders
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [16]:
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF

# Assuming you have a dataloader named 'dataloader'
for batch in dataloader:
    input_images, target_images, params = batch
    break  # Take the first batch for simplicity

In [17]:
# Convert tensors to PIL images for visualization
print(input_images[0].shape)
input_image_pil = TF.to_pil_image(input_images[0])
target_image_pil = TF.to_pil_image(target_images[0])

# Plotting the images
plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)
plt.title("Input Image")
plt.imshow(input_image_pil, cmap="gray")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title("Target Image")
plt.imshow(target_image_pil, cmap="gray")
plt.axis('off')

plt.show()

torch.Size([6, 512, 512])


ValueError: pic should not have > 4 channels. Got 6 channels.

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class UNetLike(nn.Module):
    def __init__(self):
        super(UNetLike, self).__init__()

        # Encoder
        self.down_conv1 = self.conv_block(6, 16)
        self.down_conv2 = self.conv_block(16, 32)
        self.down_conv3 = self.conv_block(32, 64)
        self.down_conv4 = self.conv_block(64, 128)
        self.down_conv5 = self.conv_block(128, 256)
        #self.down_conv6 = self.conv_block(256, 512)

        # Decoder
        #self.up_trans_1 = self.up_transpose(512, 256)
        #self.up_conv1 = self.conv_block(512, 256)
        self.up_trans_2 = self.up_transpose(256, 128)
        self.up_conv2 = self.conv_block(256, 128)
        self.up_trans_3 = self.up_transpose(128, 64)
        self.up_conv3 = self.conv_block(128, 64)
        self.up_trans_4 = self.up_transpose(64, 32)
        self.up_conv4 = self.conv_block(64, 32)
        self.up_trans_5 = self.up_transpose(32, 16)
        self.up_conv5 = self.conv_block(32, 16)

        # Final output layer
        self.out = nn.Conv2d(16, 1, kernel_size=3, stride=1, padding=1)
        self.sigmoid = nn.Sigmoid()

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1).to(device),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1).to(device),
            nn.ReLU(inplace=True),
        )

    def up_transpose(self, in_channels, out_channels):
        return nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

    def forward(self, x):
        # Encoder path
        x1 = self.down_conv1(x)
        x2 = self.down_conv2(nn.MaxPool2d(kernel_size=2, stride=2)(x1))
        x3 = self.down_conv3(nn.MaxPool2d(kernel_size=2, stride=2)(x2))
        x4 = self.down_conv4(nn.MaxPool2d(kernel_size=2, stride=2)(x3))
        x5 = self.down_conv5(nn.MaxPool2d(kernel_size=2, stride=2)(x4))
        #x6 = self.down_conv6(nn.MaxPool2d(kernel_size=2, stride=2)(x5))

        # Decoder path
        #x = self.up_trans_1(x6)  # New layer
        #x = self.up_conv1(torch.cat([x, x5], 1))

        x = self.up_trans_2(x5)
        x = self.up_conv2(torch.cat([x, x4], 1))

        x = self.up_trans_3(x)
        x = self.up_conv3(torch.cat([x, x3], 1))

        x = self.up_trans_4(x)
        x = self.up_conv4(torch.cat([x, x2], 1))

        x = self.up_trans_5(x)
        x = self.up_conv5(torch.cat([x, x1], 1))

        x = self.out(x)
        x = self.sigmoid(x)
        return x

In [20]:
from typing import Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchgeometry.image import get_gaussian_kernel2d

class SSIM(nn.Module):
    r"""Creates a criterion that measures the Structural Similarity (SSIM)
    index between each element in the input `x` and target `y`.

    The index can be described as:

    .. math::

      \text{SSIM}(x, y) = \frac{(2\mu_x\mu_y+c_1)(2\sigma_{xy}+c_2)}
      {(\mu_x^2+\mu_y^2+c_1)(\sigma_x^2+\sigma_y^2+c_2)}

    where:
      - :math:`c_1=(k_1 L)^2` and :math:`c_2=(k_2 L)^2` are two variables to
        stabilize the division with weak denominator.
      - :math:`L` is the dynamic range of the pixel-values (typically this is
        :math:`2^{\#\text{bits per pixel}}-1`).

    the loss, or the Structural dissimilarity (DSSIM) can be finally described
    as:

    .. math::

      \text{loss}(x, y) = \frac{1 - \text{SSIM}(x, y)}{2}

    Arguments:
        window_size (int): the size of the kernel.
        max_val (float): the dynamic range of the images. Default: 1.
        reduction (str, optional): Specifies the reduction to apply to the
         output: 'none' | 'mean' | 'sum'. 'none': no reduction will be applied,
         'mean': the sum of the output will be divided by the number of elements
         in the output, 'sum': the output will be summed. Default: 'none'.

    Returns:
        Tensor: the ssim index.

    Shape:
        - Input: :math:`(B, C, H, W)`
        - Target :math:`(B, C, H, W)`
        - Output: scale, if reduction is 'none', then :math:`(B, C, H, W)`

    Examples::

        >>> input1 = torch.rand(1, 4, 5, 5)
        >>> input2 = torch.rand(1, 4, 5, 5)
        >>> ssim = tgm.losses.SSIM(5, reduction='none')
        >>> loss = ssim(input1, input2)  # 1x4x5x5
    """

    def __init__(
            self,
            window_size: int,
            reduction: str = 'none',
            max_val: float = 1.0) -> None:
        super(SSIM, self).__init__()
        self.window_size: int = window_size
        self.max_val: float = max_val
        self.reduction: str = reduction

        self.window: torch.Tensor = get_gaussian_kernel2d(
            (window_size, window_size), (1.5, 1.5))
        self.padding: int = self.compute_zero_padding(window_size)

        self.C1: float = (0.01 * self.max_val) ** 2
        self.C2: float = (0.03 * self.max_val) ** 2

    @staticmethod
    def compute_zero_padding(kernel_size: int) -> int:
        """Computes zero padding."""
        return (kernel_size - 1) // 2

    def filter2D(
            self,
            input: torch.Tensor,
            kernel: torch.Tensor,
            channel: int) -> torch.Tensor:
        return F.conv2d(input, kernel, padding=self.padding, groups=channel)

    def forward(self, img1: torch.Tensor, img2: torch.Tensor) -> torch.Tensor:
        if not torch.is_tensor(img1):
            raise TypeError("Input img1 type is not a torch.Tensor. Got {}"
                            .format(type(img1)))
        if not torch.is_tensor(img2):
            raise TypeError("Input img2 type is not a torch.Tensor. Got {}"
                            .format(type(img2)))
        if not len(img1.shape) == 4:
            raise ValueError("Invalid img1 shape, we expect BxCxHxW. Got: {}"
                             .format(img1.shape))
        if not len(img2.shape) == 4:
            raise ValueError("Invalid img2 shape, we expect BxCxHxW. Got: {}"
                             .format(img2.shape))
        if not img1.shape == img2.shape:
            raise ValueError("img1 and img2 shapes must be the same. Got: {}"
                             .format(img1.shape, img2.shape))
        if not img1.device == img2.device:
            raise ValueError("img1 and img2 must be in the same device. Got: {}"
                             .format(img1.device, img2.device))
        if not img1.dtype == img2.dtype:
            raise ValueError("img1 and img2 must be in the same dtype. Got: {}"
                             .format(img1.dtype, img2.dtype))
        # prepare kernel
        b, c, h, w = img1.shape
        tmp_kernel: torch.Tensor = self.window.to(img1.device).to(img1.dtype)
        kernel: torch.Tensor = tmp_kernel.repeat(c, 1, 1, 1)

        # compute local mean per channel
        mu1: torch.Tensor = self.filter2D(img1, kernel, c)
        mu2: torch.Tensor = self.filter2D(img2, kernel, c)

        mu1_sq = mu1.pow(2)
        mu2_sq = mu2.pow(2)
        mu1_mu2 = mu1 * mu2

        # compute local sigma per channel
        sigma1_sq = self.filter2D(img1 * img1, kernel, c) - mu1_sq
        sigma2_sq = self.filter2D(img2 * img2, kernel, c) - mu2_sq
        sigma12 = self.filter2D(img1 * img2, kernel, c) - mu1_mu2

        ssim_map = ((2 * mu1_mu2 + self.C1) * (2 * sigma12 + self.C2)) / \
            ((mu1_sq + mu2_sq + self.C1) * (sigma1_sq + sigma2_sq + self.C2))

        loss = torch.clamp(1. - ssim_map, min=0, max=1) / 2.

        if self.reduction == 'mean':
            loss = torch.mean(loss)
        elif self.reduction == 'sum':
            loss = torch.sum(loss)
        elif self.reduction == 'none':
            pass
        return loss

def ssim(
        img1: torch.Tensor,
        img2: torch.Tensor,
        window_size: int,
        reduction: str = 'none',
        max_val: float = 1.0) -> torch.Tensor:
    r"""Function that measures the Structural Similarity (SSIM) index between
    each element in the input `x` and target `y`.

    See :class:`torchgeometry.losses.SSIM` for details.
    """
    return SSIM(window_size, reduction, max_val)(img1, img2)

In [21]:
class SSIMLoss(nn.Module):
    def __init__(self, window_size: int = 3, reduction: str = 'mean', max_val: float = 1.0):
        super(SSIMLoss, self).__init__()
        self.ssim = SSIM(window_size, reduction, max_val)

    def forward(self, img1: torch.Tensor, img2: torch.Tensor) -> torch.Tensor:
        return self.ssim(img1, img2)

In [22]:
def get_person_x_y(param):
    pose_start_idx = param.find("person pose (x,y,z,rot x, rot y, rot z) = ")
    if pose_start_idx != -1:
        # Check if the substring is found
        if pose_start_idx != -1:
            # Extract the relevant part of the string
            pose_str = param[pose_start_idx:]

            # Find the first "=" character
            equal_sign_idx = pose_str.find("=")

            # Extract the values after the "=" character
            values_str = pose_str[equal_sign_idx+1:].strip()

            # Split the values and convert them to integers
            values = [int(val) for val in values_str.split()[:2]]

            x_value = values[0]
            y_value = values[1]
    else:
        # No person in the picture = no additional loss
        x_value = 11
        y_value = 11

    return x_value, y_value

In [23]:
class SecondaryLoss(nn.Module):
    def __init__(self, window_size: int = 3, reduction: str = 'mean', max_val: float = 1.0, scale_factor: int = 10):
        super(SecondaryLoss, self).__init__()
        self.scale_factor = scale_factor
        self.ssim = SSIM(window_size, reduction, max_val)

    def forward(self, img1: torch.Tensor, img2: torch.Tensor, params) -> torch.Tensor:
        # Get the batch size
        batch_size = img1.size(0)
        x_values, y_values = zip(*[get_person_x_y(p) for p in params])

        # Identify indices of images with a person
        valid_indices = [i for i, (x_val, y_val) in enumerate(zip(x_values, y_values)) if x_val != 11 and y_val != 11]

        cropped_out = []
        cropped_gt = []
        for i in valid_indices:
            # Compute the positions for cropping for each element in the batch
            x_pos = 17 * (-x_values[i])
            y_pos = 17 * (-y_values[i])

            cropped_out.append(TF.crop(img1[i], 256 + x_pos - 25, 256 + y_pos - 25, 50, 50))
            cropped_gt.append(TF.crop(img2[i], 256 + x_pos - 25, 256 + y_pos - 25, 50, 50))

        # Stack the cropped results to form batches again
        cropped_out = torch.stack(cropped_out)
        cropped_gt = torch.stack(cropped_gt)

        # Calculate SSIM loss on the cropped batches and scale
        return self.ssim(cropped_out, cropped_gt) * self.scale_factor

In [ ]:
from torch.optim.lr_scheduler import StepLR

# Create the model
model = UNetLike()

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# SSIM loss function and optimizer
ssim_loss_fn = SSIMLoss()
secondary_loss_fn = SecondaryLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00015)
scheduler = StepLR(optimizer, step_size=3, gamma=0.5)

# Training loop
num_epochs = 5
best_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, targets, params in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        inputs, targets = inputs.to(device), targets.to(device)
        # Forward pass
        outputs = model(inputs)

        # Calculate the SSIM loss
        loss = ssim_loss_fn(outputs, targets)
        secondary_loss = secondary_loss_fn(outputs, targets, params)
        full_loss = loss + secondary_loss #https://stackoverflow.com/questions/53994625/how-can-i-process-multi-loss-in-pytorch
        print(loss, secondary_loss)
        # Backward and optimize
        optimizer.zero_grad()
        full_loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print the average loss for the epoch
    average_loss = running_loss / len(dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, SSIM Loss: {average_loss}")

    scheduler.step()

    if average_loss < best_loss:
        best_loss = average_loss

        # Save the model with the best validation loss
        torch.save(model.state_dict(), f"cnn_test_input/models/model{epoch + 1}.pth")

Epoch 1/5:   0%|          | 0/62 [00:00<?, ?it/s]

tensor(0.2372, grad_fn=<MeanBackward0>) tensor(2.6716, grad_fn=<MulBackward0>)


Epoch 1/5:   2%|▏         | 1/62 [01:01<1:02:28, 61.46s/it]

tensor(0.2061, grad_fn=<MeanBackward0>) tensor(2.6668, grad_fn=<MulBackward0>)


Epoch 1/5:   3%|▎         | 2/62 [02:02<1:01:10, 61.17s/it]

...

tensor(0.2453, grad_fn=<MeanBackward0>) tensor(2.7772, grad_fn=<MulBackward0>)


Epoch 1/5:  10%|▉         | 6/62 [06:29<1:02:33, 67.03s/it]

tensor(0.1954, grad_fn=<MeanBackward0>) tensor(2.4000, grad_fn=<MulBackward0>)


Epoch 1/5:  11%|█▏        | 7/62 [07:35<1:01:06, 66.66s/it]

tensor(0.2252, grad_fn=<MeanBackward0>) tensor(2.6570, grad_fn=<MulBackward0>)


In [ ]:
saved_model_path = f"cnn_test_input/models/model10.pth"
model = UNetLike()
state_dict = torch.load(saved_model_path)
model.load_state_dict(state_dict)
model.to(device)

<h3> Error in this cell, if you have time please look into printing a single image from a forward pass </h3>

In [ ]:
batch_size = 64
dataloader_single = DataLoader(dataset, batch_size=batch_size, shuffle=True)

for inputs, targets, _ in dataloader_single:
    input_images = inputs
    target_images = targets
    inputs = input_images.to(device)
    outputs = model(inputs)
    break

print(input_images.shape)
print(target_images.shape)
print(outputs.shape)

In [ ]:
for i in range(len(input_images)):
    input_image_pil = TF.to_pil_image(input_images[i])
    target_image_pil = TF.to_pil_image(target_images[i])
    output_iamge_pil = TF.to_pil_image(outputs[i])

    # Plotting the images
    plt.figure(figsize=(12, 4))  # Increase the width to accommodate three images

    plt.subplot(1, 3, 1)
    plt.title("Input Image")
    plt.imshow(input_image_pil)
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.title("Target Image")
    plt.imshow(target_image_pil)
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.title("Output")
    plt.imshow(output_iamge_pil)
    plt.axis('off')

    plt.show()